# Load Azure Daily Usage for Month to Match Invoice

The API we are using reports daily or hourly usage based on a query of the date the usage was reported. 

Since there is some delay between when the usage occured and when the usage is reported, it is important to query for a day or so *before* the first day of the invoice billing period to make sure all usage is picked up. 

This notebook analyzes an invoice with a billing period from 6/12/2018 to 7/11/2018, so we will query reported usage between 6/10/2018 to 7/14/2018 and filter down the records returned to the 6/12/2018 to 7/11/2018 period based on the actual usage date. 

In [1]:
import adal
from msrestazure.azure_active_directory import AADTokenCredentials
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv()) 
import pandas as pd
import numpy as np

In [2]:
import requests
 
# Parameters need for API
subscription = os.getenv("SUBSCRIPTION")
tenant = os.getenv("TENANT")
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
offer = 'MS-AZR-0003P'
currency = 'USD'
locale = 'en-US'
region = 'US'

In [3]:
def authenticate_client_key(tenant, client_id, client_secret):
    """
    Authenticate using service principal w/ key.
    """
    authority_host_uri = 'https://login.microsoftonline.com'
    authority_uri = authority_host_uri + '/' + tenant
    resource_uri = 'https://management.core.windows.net/'
    
    context = adal.AuthenticationContext(authority_uri, api_version=None)
    mgmt_token = context.acquire_token_with_client_credentials(resource_uri, client_id, client_secret)
    credentials = AADTokenCredentials(mgmt_token, client_id)

    return credentials


In [4]:
credentials = authenticate_client_key(tenant, client_id, client_secret)
access_token = credentials.token.get('access_token')

In [5]:
azure_mgmt_uri = 'https://management.azure.com:443/subscriptions/{subscriptionId}'.format(subscriptionId = subscription)
uri_str = "{azure_mgmt_uri}/providers/Microsoft.Commerce/UsageAggregates?" + \
    "api-version=2015-06-01-preview&" + \
    "aggregationGranularity=Daily&" + \
    "reportedstartTime=2018-06-10+00%3a00%3a00Z&" + \
    "reportedEndTime=2018-07-14+00%3a00%3a00Z"
usage_url = uri_str.format(azure_mgmt_uri = azure_mgmt_uri)
usage_url

'https://management.azure.com:443/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a87549/providers/Microsoft.Commerce/UsageAggregates?api-version=2015-06-01-preview&aggregationGranularity=Daily&reportedstartTime=2018-06-10+00%3a00%3a00Z&reportedEndTime=2018-07-14+00%3a00%3a00Z'

In [6]:
response = requests.get(usage_url, allow_redirects=False, headers = {'Authorization': 'Bearer %s' %access_token})
usage = response.json()

In [7]:
# pull the 'properties' key from each usage record and create a dataframe
df_daily_usage_api = pd.DataFrame([x['properties'] for x in usage['value']])
df_daily_usage_api

,infoFields,instanceData,meterCategory,meterId,meterName,meterRegion,meterSubCategory,quantity,subscriptionId,unit,usageEndTime,usageStartTime
0,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Storage,e9549cbe-02d9-4213-b4be-22d6dfe8a3af,Premium Storage - Page Blob/P10 (Units),US West,Locally Redundant,0.001389,3e6b71a1-1c47-4188-a4dc-793259a87549,Units,2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
1,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Data Management,9cb0bde8-bc0d-468c-8423-a25fe06779d3,Standard IO - Table Write Operation Units (in ...,NaN,NaN,0.000500,3e6b71a1-1c47-4188-a4dc-793259a87549,"10,000s",2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
2,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Networking,9995d93a-7d35-4d3f-9c69-7a7fea447ef4,Data Transfer Out (GB),Zone 1,NaN,0.059841,3e6b71a1-1c47-4188-a4dc-793259a87549,GB,2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
3,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Virtual Machines,d101de3e-ae70-48bb-8605-64fcd0a3ce8f,Compute Hours,US West,Standard_D4_v3 VM (Windows),0.766682,3e6b71a1-1c47-4188-a4dc-793259a87549,Hours,2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
4,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Data Management,b9e5e77c-a0b3-4a2c-9b8b-57fa54f31c52,Standard IO - Table Batch Write Operation Unit...,NaN,NaN,0.000100,3e6b71a1-1c47-4188-a4dc-793259a87549,"10,000s",2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
5,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Networking,d54686f0-77ff-43f3-9e7c-2099030d32a7,DNS Queries (1M),NaN,DNS,0.000804,3e6b71a1-1c47-4188-a4dc-793259a87549,1M Queries,2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
6,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Networking,f114cb19-ea64-40b5-bcd7-aee474b62853,IP Address Hours,NaN,Public IP Addresses,0.900000,3e6b71a1-1c47-4188-a4dc-793259a87549,Hours,2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
7,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Data Management,c80a3636-2edb-4248-bcb1-04ef818a75ac,Standard IO - Disk Write Operation Units (in 1...,NaN,NaN,0.009300,3e6b71a1-1c47-4188-a4dc-793259a87549,"10,000s",2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
8,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Data Management,923978e1-fd3f-4bd5-a798-f4b533057e46,Standard IO - Block Blob Delete Operation Unit...,NaN,NaN,0.004500,3e6b71a1-1c47-4188-a4dc-793259a87549,"10,000s",2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
9,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Networking,32c3ebec-1646-49e3-8127-2cafbd3a04d8,Data Transfer In (GB),Zone 1,NaN,1.619603,3e6b71a1-1c47-4188-a4dc-793259a87549,GB,2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00


In [8]:
df_daily_usage_api.dtypes

infoFields           object
instanceData         object
meterCategory        object
meterId              object
meterName            object
meterRegion          object
meterSubCategory     object
quantity            float64
subscriptionId       object
unit                 object
usageEndTime         object
usageStartTime       object
dtype: object

In [9]:
for col in ['meterId', 'meterCategory', 'meterRegion', 'meterName', 'meterSubCategory', 'subscriptionId', 'unit', 'instanceData']:
    df_daily_usage_api[col] = df_daily_usage_api[col].astype('category')
    
df_daily_usage_api.dtypes

infoFields            object
instanceData        category
meterCategory       category
meterId             category
meterName           category
meterRegion         category
meterSubCategory    category
quantity             float64
subscriptionId      category
unit                category
usageEndTime          object
usageStartTime        object
dtype: object

In [10]:
df_daily_usage_api['usageEndTime'] = pd.to_datetime(df_daily_usage_api['usageEndTime'])
df_daily_usage_api['usageStartTime'] = pd.to_datetime(df_daily_usage_api['usageStartTime'])
df_daily_usage_api.dtypes

infoFields                  object
instanceData              category
meterCategory             category
meterId                   category
meterName                 category
meterRegion               category
meterSubCategory          category
quantity                   float64
subscriptionId            category
unit                      category
usageEndTime        datetime64[ns]
usageStartTime      datetime64[ns]
dtype: object

In [11]:
df_daily_usage_api

,infoFields,instanceData,meterCategory,meterId,meterName,meterRegion,meterSubCategory,quantity,subscriptionId,unit,usageEndTime,usageStartTime
0,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Storage,e9549cbe-02d9-4213-b4be-22d6dfe8a3af,Premium Storage - Page Blob/P10 (Units),US West,Locally Redundant,0.001389,3e6b71a1-1c47-4188-a4dc-793259a87549,Units,2018-06-13,2018-06-12
1,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Data Management,9cb0bde8-bc0d-468c-8423-a25fe06779d3,Standard IO - Table Write Operation Units (in ...,NaN,NaN,0.000500,3e6b71a1-1c47-4188-a4dc-793259a87549,"10,000s",2018-06-13,2018-06-12
2,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Networking,9995d93a-7d35-4d3f-9c69-7a7fea447ef4,Data Transfer Out (GB),Zone 1,NaN,0.059841,3e6b71a1-1c47-4188-a4dc-793259a87549,GB,2018-06-13,2018-06-12
3,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Virtual Machines,d101de3e-ae70-48bb-8605-64fcd0a3ce8f,Compute Hours,US West,Standard_D4_v3 VM (Windows),0.766682,3e6b71a1-1c47-4188-a4dc-793259a87549,Hours,2018-06-13,2018-06-12
4,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Data Management,b9e5e77c-a0b3-4a2c-9b8b-57fa54f31c52,Standard IO - Table Batch Write Operation Unit...,NaN,NaN,0.000100,3e6b71a1-1c47-4188-a4dc-793259a87549,"10,000s",2018-06-13,2018-06-12
5,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Networking,d54686f0-77ff-43f3-9e7c-2099030d32a7,DNS Queries (1M),NaN,DNS,0.000804,3e6b71a1-1c47-4188-a4dc-793259a87549,1M Queries,2018-06-13,2018-06-12
6,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Networking,f114cb19-ea64-40b5-bcd7-aee474b62853,IP Address Hours,NaN,Public IP Addresses,0.900000,3e6b71a1-1c47-4188-a4dc-793259a87549,Hours,2018-06-13,2018-06-12
7,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Data Management,c80a3636-2edb-4248-bcb1-04ef818a75ac,Standard IO - Disk Write Operation Units (in 1...,NaN,NaN,0.009300,3e6b71a1-1c47-4188-a4dc-793259a87549,"10,000s",2018-06-13,2018-06-12
8,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Data Management,923978e1-fd3f-4bd5-a798-f4b533057e46,Standard IO - Block Blob Delete Operation Unit...,NaN,NaN,0.004500,3e6b71a1-1c47-4188-a4dc-793259a87549,"10,000s",2018-06-13,2018-06-12
9,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Networking,32c3ebec-1646-49e3-8127-2cafbd3a04d8,Data Transfer In (GB),Zone 1,NaN,1.619603,3e6b71a1-1c47-4188-a4dc-793259a87549,GB,2018-06-13,2018-06-12


In [12]:
len(df_daily_usage_api)

613

In [17]:
# 6/12/2018 to 7/11/2018
import datetime
from_date = pd.Timestamp(datetime.date(2018, 6, 12))
to_date = pd.Timestamp(datetime.date(2018, 7, 11))
df_invoice_daily_usage = df_daily_usage_api.loc[(df_daily_usage_api['usageStartTime'] >= from_date) & (df_daily_usage_api['usageStartTime'] <= to_date)]

In [18]:
len(df_invoice_daily_usage)

578

In [19]:
df_invoice_daily_usage['usageEndTime'].unique()

array(['2018-06-13T00:00:00.000000000', '2018-06-14T00:00:00.000000000',
       '2018-06-15T00:00:00.000000000', '2018-06-16T00:00:00.000000000',
       '2018-06-17T00:00:00.000000000', '2018-06-18T00:00:00.000000000',
       '2018-06-19T00:00:00.000000000', '2018-06-20T00:00:00.000000000',
       '2018-06-21T00:00:00.000000000', '2018-06-22T00:00:00.000000000',
       '2018-06-23T00:00:00.000000000', '2018-06-24T00:00:00.000000000',
       '2018-06-25T00:00:00.000000000', '2018-06-26T00:00:00.000000000',
       '2018-06-27T00:00:00.000000000', '2018-06-28T00:00:00.000000000',
       '2018-06-29T00:00:00.000000000', '2018-06-30T00:00:00.000000000',
       '2018-07-01T00:00:00.000000000', '2018-07-02T00:00:00.000000000',
       '2018-07-03T00:00:00.000000000', '2018-07-04T00:00:00.000000000',
       '2018-07-05T00:00:00.000000000', '2018-07-06T00:00:00.000000000',
       '2018-07-07T00:00:00.000000000', '2018-07-08T00:00:00.000000000',
       '2018-07-09T00:00:00.000000000', '2018-07-10

In [21]:
# pickle everything
import pickle
pickle.dump( df_invoice_daily_usage, open( "df_invoice_daily_usage.p", "wb" ) )
pickle.dump( df_daily_usage_api, open( "df_daily_usage_api.p", "wb" ) )